# Storing JSON Documents in Db2
Updated: 2019-09-14

### Load Db2 Extensions and Connect to the Database
The `connection` notebook contains the `CONNECT` statement which allows access to the `SAMPLE` database. If you need to modify the connection information, edit the `connection.ipynb` notebook.

In [ ]:
%run ../db2.ipynb
%run ../connection.ipynb

## Inserting and Retrieving JSON Documents
Inserting a JSON value into a Db2 table can be done through a variety of methods including `LOAD`. In the previous section, the Db2 `IMPORT` command was used to move character JSON data into a table. If the Db2 column has been defined as a character field, you can use the `INSERT` statement without any additional modification.

In [ ]:
%%sql -q
DROP TABLE CUSTOMERS;
CREATE TABLE CUSTOMERS 
(
  CUSTOMER_ID INT,
  CUSTOMER_INFO VARCHAR(2000)
);

INSERT INTO CUSTOMERS VALUES 
(
  1,
  '{"customerid": 100001,
    "identity": 
      {
        "firstname": "Kelly",
        "lastname" : "Gilmore",
        "birthdate": "1973-08-25"
      }
   }'
);

### JSON_TO_BSON and BSON_TO_JSON
If you decide to store the data in binary format, you must use the `JSON_TO_BSON` function to convert the JSON into the proper format. You also have the option of using an external BSON library to convert the string and insert the value directly into the column (i.e. Db2 is not involved in the conversion). 

In [ ]:
%%sql -q
DROP TABLE CUSTOMERS;
CREATE TABLE CUSTOMERS 
(
  CUSTOMER_ID INT,
  CUSTOMER_INFO VARBINARY(2000)
);

INSERT INTO CUSTOMERS VALUES 
(
  1,
  JSON_TO_BSON('{"customerid": 100001,
                  "identity": 
                     {
                     "firstname": "Kelly",
                     "lastname" :  "Gilmore",
                     "birthdate": "1973-08-25"
                     }
                }')
);

To retrieve an entire JSON document from a character field, you can use a standard `SELECT` statement. If the field is in BSON format, you must use the `BSON_TO_JSON` function to have it converted back into a readable format.

In [ ]:
%sql -j SELECT BSON_TO_JSON(CUSTOMER_INFO) FROM CUSTOMERS

Retrieving the data requires the use of the `BSON_TO_JSON` function to convert it back to a text format.

### Invalid JSON Detection
One of the advantages of using the new Db2 JSON functions is that you can store the data as either character (JSON) strings, or as binary (BSON) data. However, if you insert a document as a JSON character string, no checking will be done against the validity of the document until you attempt to use a JSON function against it. The following example attempts to retrieve the name field from a JSON document:

In [ ]:
%sql VALUES JSON_VALUE('{"name": George}','$.name')

From a JSON format perspective, this should fail as the value `George` is not quoted and is also not a valid number.  Surprisingly, the result of the above statement will be the `NULL` value which will seem wrong at first until you realize that the default error handling clause for any ISO JSON statement is to return a `null` by default. 



If a document needs to be checked for validity during insert, then the `JSON_TO_BSON` function can be used. The following example uses the `VALUES` clause to generate an error on an invalid JSON document. 

In [ ]:
%sql VALUES JSON_TO_BSON('{"name": George}');

The Db2 `JSON_TO_BSON` function will check the structure of the JSON document to ensure it is in the proper format. You can write a simple function that can be used to check whether or not a character string is valid JSON:

In [ ]:
%%sql -d
CREATE OR REPLACE FUNCTION CHECK_JSON(JSON CLOB)
  RETURNS INTEGER
  CONTAINS SQL LANGUAGE SQL  
  DETERMINISTIC
  NO EXTERNAL ACTION
BEGIN
  DECLARE RC BOOLEAN;
  DECLARE EXIT HANDLER FOR SQLEXCEPTION RETURN(FALSE);
  SET RC = JSON_EXISTS(JSON,'$' ERROR ON ERROR);
  RETURN(TRUE);
END

The SQL to check the previous string would look like this:

In [ ]:
%%sql
VALUES 
  CASE CHECK_JSON('{"name": George}')
    WHEN FALSE THEN 'Bad JSON' 
    WHEN TRUE  THEN 'Okay JSON' 
  END;

The function can be incorporated into a table definition as part of a check constraint.

In [ ]:
%%sql -q
DROP TABLE TESTJSON;
CREATE TABLE TESTJSON 
(
  JSON_IN VARCHAR(1000) CONSTRAINT CHECK_JSON CHECK(CHECK_JSON(JSON_IN))
);

Attempting to insert an invalid JSON document would result in the following error message being returned:

In [ ]:
%sql INSERT INTO TESTJSON VALUES '{"name": George}';

The user-defined function uses the `JSON_EXISTS` function. `JSON_EXISTS` allows you to check whether or not a valid JSON key exists within a document for the provided search path. You can use the result of this function to determine if the contents of a JSON document are consistent with your expectations and to decide whether or not to take further action or retrieve the value. You can also use this function to validate that the JSON document is properly formed.

### Summary
JSON data can be stored in Db2 in either character or binary format. The actual storage format doesn't matter when using the JSON functions. If you want to store data in BSON format, you can use external BSON conversion libraries, or use the built-in `JSON_TO_BSON` function and the corresponding `BSON_TO_JSON` function to convert it back.

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]